In [121]:
import keras 
import keras_genomics
import numpy as np
import keras.layers as k1
import simdna

from keras import backend as K 
from keras.layers.core import Dropout 
from keras.layers.core import Flatten
from keras.layers import Input
from keras.engine import Layer
from keras.models import Sequential 
from keras.engine.base_layer import InputSpec
from keras.models import Model
from keras.models import load_model
from keras.initializers import Constant

from core import MCRCDropout, CustomSumPool
from recombination_train import predict_mc
from sim_train import load_data
import types
import numpy as np

from data_utils import load_model_yaml, load_recomb_data
from core import MCRCDropout, CustomSumPool
from recombination_train import predict_mc, generate_model, train
import types
import numpy as np
import importlib

from numpy.random import seed
from tensorflow import set_random_seed
from keras.callbacks import EarlyStopping, History, ModelCheckpoint

import docopt
from keras.layers import Conv1D, GlobalMaxPool1D, Dense, Dropout, MaxPool1D
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.initializers import Constant
import keras.backend as K
from keras.engine.topology import Layer
from core import MotifMirrorGradientBleeding, CustomSumPool, MCRCDropout
from keras.models import Sequential
from keras.callbacks import EarlyStopping
import tflearn
from data_utils import one_hot, train_test_val_split, reverse_complement, load_recomb_data
from keras.models import Model
from keras.layers import GlobalMaxPool1D, GaussianDropout, Lambda
from keras.regularizers import l2
from sklearn.metrics import roc_auc_score
import gzip as gz
import types
from keras.layers import BatchNormalization
from sim_train import augment_data

In [67]:
import tflearn

In [36]:
print(len(x_train) + len(x_val) + len(x_test))

35099


In [19]:
x_train, x_val, x_test, y_train, y_val, y_test = load_recomb_data(False)

In [20]:
x_train_sim, x_val_sim, x_test_sim, y_train_sim, y_val_sim, y_test_sim = load_data(False)

In [ ]:
!pwd
with open('/users/hannahgz/revcomp_experiments/HotspotFiles/params/sym_mc_drop_data_augment.json', 'r') as f:
    my_dict = json.load(f)
    train(my_dict, 1, True)

/users/hannahgz/revcomp_experiments/HotspotFiles


In [125]:
class RevCompSpatialDropout1D(Dropout): 
    def __init__(self, rate,**kwargs): 
        super(RevCompSpatialDropout1D, self).__init__(rate, **kwargs)
        self.seed = 3
        self.input_spec = InputSpec(ndim = 3)

    def _get_noise_shape(self, inputs): 
        input_shape = K.shape(inputs)
        noise_shape = (input_shape[0], 1, 1, int(self.num_input_chan/2)) 
        return noise_shape
        
    def build(self, input_shape):
        self.num_input_chan = input_shape[2]
        self.input_len = input_shape[1]
        super(RevCompSpatialDropout1D, self).build(input_shape)

    def call(self, inputs, training=None): 
        inputs_fwdandrevconcat = K.concatenate(
                tensors = [
                    inputs[:,:,None,:int(self.num_input_chan/2)],
                    inputs[:,:,None,int(self.num_input_chan/2):][:,:,:,::-1]],
                axis=2)

        if 0. < self.rate < 1.: 
            noise_shape = self._get_noise_shape(inputs)
            def dropped_inputs(): 
                dropped = K.dropout(inputs_fwdandrevconcat,
                                    self.rate, noise_shape, seed = self.seed)
                dropped = K.reshape(dropped, (-1, int(self.input_len), int(self.num_input_chan)))
                return K.concatenate(
                    tensors = [
                        dropped[:,:,:int(self.num_input_chan/2)],
                        dropped[:,:,int(self.num_input_chan/2):][:,:,::-1]],
                    axis=-1)

            return K.in_train_phase(dropped_inputs, inputs, training = training)

        return inputs
class RevCompSumPool(Layer): 
    def __init__(self, **kwargs): 
        super(RevCompSumPool, self).__init__(**kwargs)

    def build(self, input_shape):
        self.num_input_chan = input_shape[2]
        super(RevCompSumPool, self).build(input_shape)

    def call(self, inputs): 
        #divide by sqrt 2 for variance preservation
        inputs = (inputs[:,:,:int(self.num_input_chan/2)] + inputs[:,:,int(self.num_input_chan/2):][:,::-1,::-1])/(1.41421356237)
        return inputs
      
    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1], int(input_shape[2]/2))
class RevComp(Layer): 
    def __init__(self, **kwargs): 
      super(RevComp, self).__init__(**kwargs)

    def build(self, input_shape):
      super(RevComp, self).build(input_shape)

    def call(self, inputs): 
      return inputs[:,::-1,::-1]
      
    def compute_output_shape(self, input_shape):
      return input_shape

def predict_with_uncertainty(f, x, no_classes, n_iter=100):
    result = np.zeros((n_iter,) + (x.shape[0], no_classes) )

    for i in range(n_iter):
        result[i,:, :] = f((x, 1))[0]

    prediction = result.mean(axis=0)
    uncertainty = result.std(axis=0)
    return prediction

def spearmanr_all(y_test, model, test_batch_generator): 
#     model = load_model(name, custom_objects)
    f2 = K.function([model.layers[0].input, K.learning_phase()],
                    [model.layers[-1].output])
    
    y_pred = np.concatenate(np.array([predict_with_uncertainty(f2, test_batch_generator[i][0], 1) for i in range(len(test_batch_generator))]), axis = 0)
    rho, pval = spearmanr(y_test, y_pred)
    return rho

custom_objects = {'RevCompConv1D':keras_genomics.layers.RevCompConv1D,
                  'RevCompSumPool':RevCompSumPool,
                  'MCRCDropout':MCRCDropout,
                  'RevCompSpatialDropout1D': RevCompSpatialDropout1D,
                  'RevComp':RevComp}


def predict_with_uncertainty(f, x, no_classes, n_iter=100):
    result = np.zeros((n_iter,) + (x.shape[0], no_classes) )

    for i in range(n_iter):
        result[i,:, :] = f((x, 1))[0]

    prediction = result.mean(axis=0)
    uncertainty = result.std(axis=0)
    return prediction  

def evaluate(model, x, y): 
    y_pred = model.predict(x)
    auroc = roc_auc_score(y, model.predict(x)) 
    auprc = average_precision_score(y, model.predict(x))
    return auroc, auprc

def evaluate_dropout(model, x, y): 
    f2 = K.function([model.layers[0].input, K.learning_phase()],
               [model.layers[-1].output])
    y_pred = predict_with_uncertainty(f2, x, 1, n_iter = 100) 
    print(y_pred.shape)
    auroc = roc_auc_score(y, y_pred) 
    auprc = average_precision_score(y, y_pred)
    return auroc, auprc
  
def predict_dropout(model, x, y, n_preds): 
    f2 = K.function([model.layers[0].input, K.learning_phase()],
                    [model.layers[-1].output])
    y_pred = predict_with_uncertainty(f2, x, 1, n_iter = n_preds) 
    acc = binary_accuracy(tflearn.data_utils.to_categorical(y_test,2), y_pred)
    return acc

def predict_mc(model, x_pred, y_pred, n_preds):
    y_pred = np.mean([model.predict(x_pred) for i in range(n_preds)], axis=0)
    acc = binary_accuracy(tflearn.data_utils.to_categorical(y_test,2), y_pred)
    return acc

def binary_accuracy(y_true, y_pred):
    return np.mean(np.equal(y_true, np.round(y_pred)))

kernel_size = 15
filters= 15
input_length = 1000
import json


In [60]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
def test_sim(data):
    num_conv = data["num_conv"]
    activation = data["activation"]
    custom_init = data["custom_init"]
    dataset = data["dataset"]
    augment =  data["augment"]
    dropout = data["dropout"]
    dropout_rate = data["dropout_rate"]
    filename = data["filename"]
    filters = data["filters"]
    kernel_size = data["kernel_size"]
    mc_dropout = data["mc_dropout"]
    num_epochs = data["num_epochs"]
    patience = data["patience"]
    pool_size = data["pool_size"]
    pooling = data["pooling"]
    rev_comp = data["rev_comp"]
    seed_num = (int)(np.random.uniform(1000, 10000))
    data["seed_num"] = seed_num
    siamese = data["siamese"]
    spatial_dropout = data["spatial_dropout"]
    rc_spatial_dropout = data["rc_spatial_dropout"]
    strides = data["strides"]
    units = data["units"]
        
    seed(seed_num)
    set_random_seed(seed_num)
    model = keras.models.Sequential()
    
    for i in range(num_conv): 
        if rev_comp: 
            model.add(keras_genomics.layers.RevCompConv1D(filters=filters, 
                                                          kernel_size=kernel_size, 
                                                          input_shape=x_train_sim.shape[1:], 
                                                          padding="same"))
        else: 
            model.add(k1.Conv1D(filters=filters, kernel_size=kernel_size,
                          input_shape=x_train_sim.shape[1:],
                          padding="same")) 
            
        model.add(k1.core.Activation(activation))
        
        #No dropout before the pooling layer
        if i != num_conv - 1: 
            if dropout: 
                model.add(k1.Dropout(dropout_rate))
            elif spatial_dropout: 
                model.add(k1.SpatialDropout1D(dropout_rate))
            elif rc_spatial_dropout:
                model.add(RevCompSpatialDropout1D(dropout_rate)) 
            elif mc_dropout: 
                model.add(MCRCDropout(dropout_rate))
    divisor = 1
    #Only needed with rc model
    if rev_comp: 
        model.add(RevCompSumPool())
        divisor = 2
    
    #Pooling layers
    if pooling == 'max': 
        model.add(k1.pooling.MaxPooling1D(pool_size = pool_size,padding = "same", strides = strides))
    elif pooling == 'avg':
        model.add(k1.pooling.AveragePooling1D(pool_size = pool_size, padding = "same", strides = strides))
    
    model.add(Flatten())
    
    #Fully-connected layers 
    if custom_init: 
        model.add(keras_genomics.layers.core.Dense(2, activation="softmax", kernel_initializer= Constant(np.array([[1]*(filters//divisor), [1]*(filters//divisor)])),
            bias_initializer=Constant(np.array([1, -1]))))
#         model.add(Dense(2, activation="softmax", kernel_initializer=Constant(np.array([[1]*(nn_params["input_filters"]//divisor), [1]*(nn_params["input_filters"]//divisor)])),
#         bias_initializer=Constant(np.array([1, -1]))))
    model.add(keras_genomics.layers.core.Dense(units = 1))  
    
    if siamese: 
        main_input = Input(shape=x_train_sim.shape[1:])
        rev_input = Input(shape=x_train_sim.shape[1:])
        rev_input = RevComp()(main_input)
        main_output = model(main_input)
        rev_output = model(rev_input)
        avg = k1.Average()([main_output, rev_output])
        model = Model(inputs = main_input, outputs = avg)

    model.summary()
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    early_stopping_callback = keras.callbacks.EarlyStopping(
                                  monitor='val_loss',
                                  patience= patience,
                                  restore_best_weights=True)
    history_model = model.fit(x_train_sim, y_train_sim, validation_split=0.2,  
                        callbacks= [early_stopping_callback], batch_size=100,  epochs=num_epochs)
    model.set_weights(early_stopping_callback.best_weights)

        
    model.save("/users/hannahgz/revcomp_experiments/HotspotFiles/HotspotResults/%s_%s.h5" % (filename,str(seed_num)))

    rho = 0
    if dropout or mc_dropout or spatial_dropout or rc_spatial_dropout: 
        auroc, auprc = evaluate_dropout(model, x_test_sim, y_test_sim)
    else: 
        auroc, auprc = evaluate(model, x_test_sim, y_test_sim)
    
    data["auroc"] = auroc 
    data["auprc"] = auprc
    
    with open("/users/hannahgz/revcomp_experiments/HotspotFiles/HotspotResults/config_%s_%s.json" % (filename,str(seed_num)), "w") as data_file: 
        json.dump(data, data_file, indent=2)

In [139]:
def get_session():
    try:
        #use the keras session if there is one
        import keras.backend as K
        return K.get_session()
    except:
        #Warning: I haven't really tested this behaviour out...
        global _SESS 
        if _SESS is None:
            print("MAKING A SESSION")
            _SESS = tf.Session()
            _SESS.run(tf.global_variables_initializer()) 
        return _SESS

def compile_func(inputs, outputs):
    if (isinstance(inputs, list)==False):
        print("Wrapping the inputs in a list...")
        inputs = [inputs]
    assert isinstance(inputs, list)
    def func_to_return(inp):
        if len(inp) > len(inputs) and len(inputs)==1:
            print("Wrapping the inputs in a list...")
            inp = [inp]
        assert len(inp)==len(inputs),\
            ("length of provided list should be "
             +str(len(inputs))+" for tensors "+str(inputs)
             +" but got input of length "+str(len(inp)))
        feed_dict = {}
        for input_tensor, input_val in zip(inputs, inp):
            feed_dict[input_tensor] = input_val 
        sess = get_session()
        return sess.run(outputs, feed_dict=feed_dict)  
    return func_to_return

import tensorflow as tf
# grads_tensor = tf.gradients(ys=model.layers[-1].output, xs=model.input)
# grad_func = compile_func(inputs=model.input, outputs=grads_tensor)
# gradients = grad_func(batch_of_one_hot_encoded_input)

In [146]:
from statistics import mean

In [188]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
def test(data):
    num_conv = data["num_conv"]
    activation = data["activation"]
    custom_init = data["custom_init"]
    dataset = data["dataset"]
    augment =  data["augment"]
    dropout = data["dropout"]
    dropout_rate = data["dropout_rate"]
    filename = data["filename"]
    filters = data["filters"]
    kernel_size = data["kernel_size"]
    mc_dropout = data["mc_dropout"]
    num_epochs = data["num_epochs"]
    patience = data["patience"]
    pool_size = data["pool_size"]
    pooling = data["pooling"]
    rev_comp = data["rev_comp"]
    data["seed_num"] = seed_num
    siamese = data["siamese"]
    spatial_dropout = data["spatial_dropout"]
    rc_spatial_dropout = data["rc_spatial_dropout"]
    strides = data["strides"]
    units = data["units"]
        
    seed(seed_num)
    set_random_seed(seed_num)
    model = keras.models.Sequential()
    
    for i in range(num_conv): 
        if rev_comp: 
            model.add(keras_genomics.layers.RevCompConv1D(filters=filters, 
                                                          kernel_size=kernel_size, 
                                                          input_shape=x_train.shape[1:], 
                                                          padding="same"))
        else: 
            model.add(k1.Conv1D(filters=filters, kernel_size=kernel_size,
                          input_shape=x_train.shape[1:],
                          padding="same")) 
            
        model.add(k1.core.Activation("relu"))
        
        #No dropout before the pooling layer
        if i != num_conv - 1: 
            if dropout: 
                model.add(k1.Dropout(dropout_rate))
            elif spatial_dropout: 
                model.add(k1.SpatialDropout1D(dropout_rate))
            elif rc_spatial_dropout:
                model.add(RevCompSpatialDropout1D(dropout_rate)) 
            elif mc_dropout: 
                model.add(MCRCDropout(dropout_rate))
    divisor = 0
    #Only needed with rc model
    if rev_comp: 
        model.add(RevCompSumPool())
        divisor = 2
    else: 
        divisor = 1
    
    #Pooling layers
    if pooling == 'max': 
        model.add(k1.pooling.MaxPooling1D(pool_size = pool_size,padding = "same", strides = strides))
    elif pooling == 'avg':
        model.add(k1.pooling.AveragePooling1D(pool_size = pool_size, padding = "same", strides = strides))
    
    model.add(Flatten())
    
    #Fully-connected layers 
    if custom_init == "custom_init": 
        model.add(keras_genomics.layers.core.Dense(2, activation="softmax", kernel_initializer= Constant(np.array([[1]*(filters//divisor), [1]*(filters//divisor)])),
            bias_initializer=Constant(np.array([1, -1]))))
    elif custom_init == "custom_init_no_bias": 
        model.add(keras_genomics.layers.core.Dense(2, activation="softmax", kernel_initializer= Constant(np.array([[1]*(filters//divisor), [1]*(filters//divisor)]))))
    elif custom_init == "glorot_uniform": 
        model.add(keras_genomics.layers.core.Dense(2, activation="softmax", kernel_initializer = keras.initializers.glorot_uniform()))
    elif custom_init == "he_normal": 
        model.add(keras_genomics.layers.core.Dense(2, activation="softmax", kernel_initializer = keras.initializers.he_normal()))
    else: 
        model.add(keras_genomics.layers.core.Dense(2, activation = "softmax"))
    
    if siamese: 
        main_input = Input(shape=x_train.shape[1:])
        rev_input = Input(shape=x_train.shape[1:])
        rev_input = RevComp()(main_input)
        main_output = model(main_input)
        rev_output = model(rev_input)
        avg = k1.Average()([main_output, rev_output])
        model = Model(inputs = main_input, outputs = avg)

    model.summary()
    model.compile(keras.optimizers.Adam(lr=0.1, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False), 
                  loss="categorical_crossentropy", metrics=["accuracy"])
    early_stopping_callback = keras.callbacks.EarlyStopping(
                                  monitor='val_loss',
                                  patience= patience,
                                  restore_best_weights=True)
    grads_tensor = tf.gradients(ys=model.layers[-1].output, xs=model.input)
    grad_func = compile_func(inputs=model.input, outputs=grads_tensor)
    gradients = grad_func(x_train[0:500])
    print(np.average(gradients[0]))
    history_model = model.fit(x_train, tflearn.data_utils.to_categorical(y_train, 2), 
                              validation_data = (x_val, tflearn.data_utils.to_categorical(y_val,2)),
                              callbacks= [early_stopping_callback], batch_size=100,  epochs=num_epochs)
    model.set_weights(early_stopping_callback.best_weights)

        
    model.save("/users/hannahgz/revcomp_experiments/HotspotFiles/Hotspot_Results/%s_%s.h5" % (filename,str(seed_num)))

    acc = 0
    if dropout or mc_dropout or spatial_dropout or rc_spatial_dropout: 
        acc = predict_mc(model, x_test, y_test, 100)
    else: 
        acc = predict_mc(model, x_test, y_test, 1)
    
    data["acc"] = acc
    
    with open("/users/hannahgz/revcomp_experiments/HotspotFiles/Hotspot_Results/config_%s_%s.json" % (filename,str(seed_num)), "w") as data_file: 
        json.dump(data, data_file, indent=2)

In [162]:
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = '5'

In [169]:
data = {
        "filename": "rc",
        "dataset": "hotspot",
        "augment": False,
        "siamese": False,
        "rev_comp": True,
        "custom_init": False,
        "dropout": False,
        "mc_dropout": False,
        "spatial_dropout": False,
        "rc_spatial_dropout": False,
        "activation": "relu",
        "dropout_rate": 0.2,
        "num_conv": 3,
        "filters": 12,
        "kernel_size": 14,
        "pool_size": 40,
        "pooling": "max",
        "num_epochs": 100,
        "patience": 60,
        "seed_num": 1000,
        "strides": 40,
        "units": 2,
        "correlation": 0 
    }
test(data)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rev_comp_conv1d_76 (RevCompC (None, 997, 24)           684       
_________________________________________________________________
activation_223 (Activation)  (None, 997, 24)           0         
_________________________________________________________________
rev_comp_conv1d_77 (RevCompC (None, 997, 24)           4044      
_________________________________________________________________
activation_224 (Activation)  (None, 997, 24)           0         
_________________________________________________________________
rev_comp_conv1d_78 (RevCompC (None, 997, 24)           4044      
_________________________________________________________________
activation_225 (Activation)  (None, 997, 24)           0         
_________________________________________________________________
rev_comp_sum_pool_26 (RevCom (None, 997, 12)           0         
__________

In [170]:
data = {
        "filename": "rc_custom_init",
        "dataset": "hotspot",
        "augment": False,
        "siamese": False,
        "custom_init": True,
        "rev_comp": True,
        "dropout": False,
        "mc_dropout": False,
        "spatial_dropout": False,
        "rc_spatial_dropout": False,
        "activation": "relu",
        "dropout_rate": 0.2,
        "num_conv": 3,
        "filters": 12,
        "kernel_size": 14,
        "pool_size": 40,
        "pooling": "max",
        "num_epochs": 100,
        "patience": 60,
        "seed_num": 1000,
        "strides": 40,
        "units": 2,
        "correlation": 0 
    }
test(data)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rev_comp_conv1d_79 (RevCompC (None, 997, 24)           684       
_________________________________________________________________
activation_226 (Activation)  (None, 997, 24)           0         
_________________________________________________________________
rev_comp_conv1d_80 (RevCompC (None, 997, 24)           4044      
_________________________________________________________________
activation_227 (Activation)  (None, 997, 24)           0         
_________________________________________________________________
rev_comp_conv1d_81 (RevCompC (None, 997, 24)           4044      
_________________________________________________________________
activation_228 (Activation)  (None, 997, 24)           0         
_________________________________________________________________
rev_comp_sum_pool_27 (RevCom (None, 997, 12)           0         
__________

In [171]:
data = {
        "filename": "reg",
        "dataset": "hotspot",
        "augment": False,
        "siamese": False,
        "custom_init": False,
        "rev_comp": False,
        "dropout": False,
        "mc_dropout": False,
        "spatial_dropout": False,
        "rc_spatial_dropout": False,
        "activation": "relu",
        "dropout_rate": 0.2,
        "num_conv": 3,
        "filters": 12,
        "kernel_size": 14,
        "pool_size": 40,
        "pooling": "max",
        "num_epochs": 100,
        "patience": 60,
        "seed_num": 1000,
        "strides": 40,
        "units": 2,
        "correlation": 0 
    }
test(data)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_164 (Conv1D)          (None, 997, 12)           684       
_________________________________________________________________
activation_229 (Activation)  (None, 997, 12)           0         
_________________________________________________________________
conv1d_165 (Conv1D)          (None, 997, 12)           2028      
_________________________________________________________________
activation_230 (Activation)  (None, 997, 12)           0         
_________________________________________________________________
conv1d_166 (Conv1D)          (None, 997, 12)           2028      
_________________________________________________________________
activation_231 (Activation)  (None, 997, 12)           0         
_________________________________________________________________
max_pooling1d_83 (MaxPooling (None, 25, 12)            0         
__________

In [172]:
data = {
        "filename": "reg_custom_init",
        "dataset": "hotspot",
        "augment": False,
        "siamese": False,
        "custom_init": True,
        "rev_comp": False,
        "dropout": False,
        "mc_dropout": False,
        "spatial_dropout": False,
        "rc_spatial_dropout": False,
        "activation": "relu",
        "dropout_rate": 0.2,
        "num_conv": 3,
        "filters": 12,
        "kernel_size": 14,
        "pool_size": 40,
        "pooling": "max",
        "num_epochs": 100,
        "patience": 60,
        "seed_num": 1000,
        "strides": 40,
        "units": 2,
        "correlation": 0 
    }
test(data)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_167 (Conv1D)          (None, 997, 12)           684       
_________________________________________________________________
activation_232 (Activation)  (None, 997, 12)           0         
_________________________________________________________________
conv1d_168 (Conv1D)          (None, 997, 12)           2028      
_________________________________________________________________
activation_233 (Activation)  (None, 997, 12)           0         
_________________________________________________________________
conv1d_169 (Conv1D)          (None, 997, 12)           2028      
_________________________________________________________________
activation_234 (Activation)  (None, 997, 12)           0         
_________________________________________________________________
max_pooling1d_84 (MaxPooling (None, 25, 12)            0         
__________

In [180]:
data = {
        "filename": "reg_glorot_uniform",
        "dataset": "hotspot",
        "augment": False,
        "siamese": False,
        "custom_init": "glorot_uniform",
        "rev_comp": False,
        "dropout": False,
        "mc_dropout": False,
        "spatial_dropout": False,
        "rc_spatial_dropout": False,
        "activation": "relu",
        "dropout_rate": 0.2,
        "num_conv": 3,
        "filters": 12,
        "kernel_size": 14,
        "pool_size": 40,
        "pooling": "max",
        "num_epochs": 100,
        "patience": 60,
        "seed_num": 1000,
        "strides": 40,
        "units": 2,
        "correlation": 0 
    }
test(data)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_176 (Conv1D)          (None, 997, 12)           684       
_________________________________________________________________
activation_241 (Activation)  (None, 997, 12)           0         
_________________________________________________________________
conv1d_177 (Conv1D)          (None, 997, 12)           2028      
_________________________________________________________________
activation_242 (Activation)  (None, 997, 12)           0         
_________________________________________________________________
conv1d_178 (Conv1D)          (None, 997, 12)           2028      
_________________________________________________________________
activation_243 (Activation)  (None, 997, 12)           0         
_________________________________________________________________
max_pooling1d_87 (MaxPooling (None, 25, 12)            0         
__________

In [182]:
data = {
        "filename": "reg_he_normal",
        "dataset": "hotspot",
        "augment": False,
        "siamese": False,
        "custom_init": "he_normal",
        "rev_comp": False,
        "dropout": False,
        "mc_dropout": False,
        "spatial_dropout": False,
        "rc_spatial_dropout": False,
        "activation": "relu",
        "dropout_rate": 0.2,
        "num_conv": 3,
        "filters": 12,
        "kernel_size": 14,
        "pool_size": 40,
        "pooling": "max",
        "num_epochs": 100,
        "patience": 60,
        "seed_num": 1000,
        "strides": 40,
        "units": 2,
        "correlation": 0 
    }
test(data)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_179 (Conv1D)          (None, 997, 12)           684       
_________________________________________________________________
activation_244 (Activation)  (None, 997, 12)           0         
_________________________________________________________________
conv1d_180 (Conv1D)          (None, 997, 12)           2028      
_________________________________________________________________
activation_245 (Activation)  (None, 997, 12)           0         
_________________________________________________________________
conv1d_181 (Conv1D)          (None, 997, 12)           2028      
_________________________________________________________________
activation_246 (Activation)  (None, 997, 12)           0         
_________________________________________________________________
max_pooling1d_88 (MaxPooling (None, 25, 12)            0         
__________

In [185]:
data = {
        "filename": "reg_no_bias",
        "dataset": "hotspot",
        "augment": False,
        "siamese": False,
        "custom_init": "custom_init_no_bias",
        "rev_comp": False,
        "dropout": False,
        "mc_dropout": False,
        "spatial_dropout": False,
        "rc_spatial_dropout": False,
        "activation": "relu",
        "dropout_rate": 0.2,
        "num_conv": 3,
        "filters": 12,
        "kernel_size": 14,
        "pool_size": 40,
        "pooling": "max",
        "num_epochs": 100,
        "patience": 60,
        "seed_num": 1000,
        "strides": 40,
        "units": 2,
        "correlation": 0 
    }
test(data)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_185 (Conv1D)          (None, 997, 12)           684       
_________________________________________________________________
activation_250 (Activation)  (None, 997, 12)           0         
_________________________________________________________________
conv1d_186 (Conv1D)          (None, 997, 12)           2028      
_________________________________________________________________
activation_251 (Activation)  (None, 997, 12)           0         
_________________________________________________________________
conv1d_187 (Conv1D)          (None, 997, 12)           2028      
_________________________________________________________________
activation_252 (Activation)  (None, 997, 12)           0         
_________________________________________________________________
max_pooling1d_90 (MaxPooling (None, 25, 12)            0         
__________

In [187]:
data = {
        "filename": "reg_lr_0.01",
        "dataset": "hotspot",
        "augment": False,
        "siamese": False,
        "custom_init": "",
        "rev_comp": False,
        "dropout": False,
        "mc_dropout": False,
        "spatial_dropout": False,
        "rc_spatial_dropout": False,
        "activation": "relu",
        "dropout_rate": 0.2,
        "num_conv": 3,
        "filters": 12,
        "kernel_size": 14,
        "pool_size": 40,
        "pooling": "max",
        "num_epochs": 100,
        "patience": 60,
        "seed_num": 1000,
        "strides": 40,
        "units": 2,
        "correlation": 0 
    }
test(data)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_188 (Conv1D)          (None, 997, 12)           684       
_________________________________________________________________
activation_253 (Activation)  (None, 997, 12)           0         
_________________________________________________________________
conv1d_189 (Conv1D)          (None, 997, 12)           2028      
_________________________________________________________________
activation_254 (Activation)  (None, 997, 12)           0         
_________________________________________________________________
conv1d_190 (Conv1D)          (None, 997, 12)           2028      
_________________________________________________________________
activation_255 (Activation)  (None, 997, 12)           0         
_________________________________________________________________
max_pooling1d_91 (MaxPooling (None, 25, 12)            0         
__________

In [189]:
data = {
        "filename": "reg_lr_0.1",
        "dataset": "hotspot",
        "augment": False,
        "siamese": False,
        "custom_init": "",
        "rev_comp": False,
        "dropout": False,
        "mc_dropout": False,
        "spatial_dropout": False,
        "rc_spatial_dropout": False,
        "activation": "relu",
        "dropout_rate": 0.2,
        "num_conv": 3,
        "filters": 12,
        "kernel_size": 14,
        "pool_size": 40,
        "pooling": "max",
        "num_epochs": 100,
        "patience": 60,
        "seed_num": 1000,
        "strides": 40,
        "units": 2,
        "correlation": 0 
    }
test(data)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_191 (Conv1D)          (None, 997, 12)           684       
_________________________________________________________________
activation_256 (Activation)  (None, 997, 12)           0         
_________________________________________________________________
conv1d_192 (Conv1D)          (None, 997, 12)           2028      
_________________________________________________________________
activation_257 (Activation)  (None, 997, 12)           0         
_________________________________________________________________
conv1d_193 (Conv1D)          (None, 997, 12)           2028      
_________________________________________________________________
activation_258 (Activation)  (None, 997, 12)           0         
_________________________________________________________________
max_pooling1d_92 (MaxPooling (None, 25, 12)            0         
__________

In [ ]:
for seed_num in range(1000, 10000, 1000): 
    data = {
            "filename": "rc_orig_spatial_dropout",
            "dataset": "hotspot",
            "augment": False,
            "siamese": False,
            "rev_comp": True,
            "dropout": False,
            "mc_dropout": False,
            "spatial_dropout": True,
            "rc_spatial_dropout": False,
            "activation": "relu",
            "dropout_rate": 0.2,
            "num_conv": 3,
            "filters": 12,
            "kernel_size": 14,
            "pool_size": 40,
            "pooling": "max",
            "num_epochs": 100,
            "patience": 60,
            "seed_num": seed_num,
            "strides": 40,
            "units": 2,
            "correlation": 0 
        }
    test(data)
    
    data["spatial_dropout"] = False
    data["rc_spatial_dropout"] = True 
    data["filename"] = "rc_custom_spatial_dropout"
    test(data)
    
    data["rev_comp"] = False 
    data["siamese"] = True 
    data["spatial_dropout"] = True 
    data["filename"] = "siamese_spatial_dropout"
    test(data)

# data = {
#         "filename": "rc_5_filters",
#         "dataset": "CTCF",
#         "augment": False,
#         "siamese": False,
#         "rev_comp": True,
#         "dropout": False,
#         "mc_dropout": False,
#         "spatial_dropout": False,
#         "rc_spatial_dropout": False,
#         "dropout_rate": 0.2,
#         "num_conv": 3,
#         "filters": 5,
#         "kernel_size": 15,
#         "pool_size": 40,
#         "pooling": "max",
#         "num_epochs": 300,
#         "patience": 60,
#         "seed_num": 1000,
#         "strides": 40,
#         "units": 100,
#         "correlation": 0 
#         }
# test(data)

# data["filename"] = "rc_rc_spatial_dropout_5_filters"
# data["spatial_dropout"] = False
# data["rc_spatial_dropout"] = True
# test(data)

# data["filename"] = "rc_mc_dropout_5_filters"
# data["rc_spatial_dropout"] = False
# data["mc_dropout"] = True
# test(data)

# data["filename"] = "rc_dropout_5_filters"
# data["mc_dropout"] = False 
# data["dropout"] = True
# test(data)

# data["filename"] = "reg_dropout_5_filters"
# data["rev_comp"] = False
# test(data)

# data["filename"] = "reg_spatial_dropout_5_filters"
# data["dropout"] = False
# data["spatial_dropout"] = True
# test(data)

# data["filename"] = "augment_spatial_dropout_5_filters"
# data["augment"] = True
# test(data)

# data["filename"] = "augment_dropout_5_filters"
# data["dropout"] = True
# data["spatial_dropout"] = False
# test(data)

# data["filename"] = "siamese_dropout_5_filters"
# data["siamese"] = True
# data["augment"] = False
# test(data)

# data["filename"] = "siamese_spatial_dropout_5_filters"
# data["dropout"] = False
# data["spatial_dropout"] = True
# test(data)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rev_comp_conv1d_10 (RevCompC (None, 997, 24)           684       
_________________________________________________________________
activation_88 (Activation)   (None, 997, 24)           0         
_________________________________________________________________
spatial_dropout1d_3 (Spatial (None, 997, 24)           0         
_________________________________________________________________
rev_comp_conv1d_11 (RevCompC (None, 997, 24)           4044      
_________________________________________________________________
activation_89 (Activation)   (None, 997, 24)           0         
_________________________________________________________________
spatial_dropout1d_4 (Spatial (None, 997, 24)           0         
_________________________________________________________________
rev_comp_conv1d_12 (RevCompC (None, 997, 24)           4044      
__________

In [13]:
data = {
        "filename": "rc",
        "dataset": "hotspot",
        "augment": False,
        "siamese": True,
        "rev_comp": False,
        "dropout": True,
        "mc_dropout": False,
        "spatial_dropout": False,
        "rc_spatial_dropout": False,
        "dropout_rate": 0.2,
        "num_conv": 3,
        "filters": 15,
        "kernel_size": 15,
        "pool_size": 40,
        "pooling": "max",
        "num_epochs": 100,
        "patience": 60,
        "seed_num": 1000,
        "strides": 40,
        "units": 100,
        "correlation": 0 
    }
test(data)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 997, 4)       0                                            
__________________________________________________________________________________________________
rev_comp_2 (RevComp)            (None, 997, 4)       0           input_3[0][0]                    
__________________________________________________________________________________________________
sequential_8 (Sequential)       (None, 1)            45396       input_3[0][0]                    
                                                                 rev_comp_2[0][0]                 
__________________________________________________________________________________________________
average_2 (Average)             (None, 1)            0           sequential_8[1][0]               
          

KeyboardInterrupt: 